# ImageNet Analyze different methods

This notebook illustrates the effect of the proposed methods on some recent networks. We compare the results based on a single image.

## Imports

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
%matplotlib inline  

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import imp
import time
import sys
import os
import argparse

import keras.backend
import keras.models

import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.tests.networks.imagenet
import innvestigate.utils.visualizations as ivis

eutils = imp.load_source("utils", "../utils.py")
imgnetutils = imp.load_source("utils_imagenet", "../utils_imagenet.py")

Using TensorFlow backend.


# Choose & load image

This analysis is based on a single test image of the ones stored in `examples/images`. Other example images can be used by loading these into the above folder.

In [3]:
# Choose the number of an image in the example folder (between 0-6)
image_n = 3

# Choose models

A full list of currently available models is available at `/innvestigate/applications/imagenet.py`.

In [4]:
# Choose a list of models
netnames = ["vgg16",
            "resnet50",
            "inception_v3",
            "densenet121"]
           

n_nets = len(netnames)

# Analysis

In [ ]:
pattern_type = "relu"
channels_first = keras.backend.image_data_format == "channels_first"
analysis_all = []
text = []

for i, netname in enumerate(netnames):
    print("Analyzing {}...".format(netname))
    
    # Build model.
    tmp = getattr(innvestigate.applications.imagenet, netname)
    net = tmp(load_weights=True, load_patterns=pattern_type)
    model = keras.models.Model(inputs=net["in"], outputs=net["out"])
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    modelp = keras.models.Model(inputs=net["in"], outputs=net["sm_out"])
    modelp.compile(optimizer="adam", loss="categorical_crossentropy")
    
    # Load image from example folder
    images, label_to_class_name = eutils.get_imagenet_data(net["image_shape"][0])
    images = images[image_n]
    
    
    color_conversion = "BGRtoRGB" if net["color_coding"] == "BGR" else None

    # Analysis.
    patterns = net["patterns"]
        
    methods = [
        # NAME             POSTPROCESSING     TITLE

        # Show input.
        ("input",                 {},                            imgnetutils.image,   "Input"),

        # Function
        ("gradient",              {},                            imgnetutils.graymap, "Gradient"),
        ("integrated_gradients",  {},                            imgnetutils.graymap, "Integrated Gradients"),

        # Signal
        ("deconvnet",             {},                            imgnetutils.bk_proj, "Deconvnet"),
        ("guided_backprop",       {},                            imgnetutils.bk_proj, "Guided Backprop"),
        ("pattern.net",           {"patterns": patterns},        imgnetutils.bk_proj, "PatterNet"),

        # Interaction
        ("pattern.attribution",   {"patterns": patterns},        imgnetutils.heatmap, "Pattern Attribution"),
        ("lrp.epsilon",           {"epsilon": 1},                imgnetutils.heatmap, "LRP Epsilon"),
        ("lrp.sequential_preset_a_flat", {"epsilon": 1},         imgnetutils.heatmap, "LRP-PresetAFlat"),
        ("lrp.sequential_preset_b_flat", {"epsilon": 1},         imgnetutils.heatmap,  "LRP-PresetBFlat"),
    ]

    # Create analyzers.
    analyzers = []
    for method in methods:
        try:
            analyzer = innvestigate.create_analyzer(method[0],
                                                      model,
                                                      **method[1])
        except innvestigate.NotAnalyzeableModelException:
            analyzer = None
        analyzers.append(analyzer)

    # Create analysis.
    analysis = np.zeros([len(analyzers)] + net["image_shape"] + [3])

    image,y = images
    image = image[None, :, :, :]
    # Predict label.
    x = imgnetutils.preprocess(image, net)
    presm = model.predict_on_batch(x)[0]
    prob = modelp.predict_on_batch(x)[0]
    y_hat = prob.argmax()
    
    # Save prediction info:
    text.append(("%s" %label_to_class_name[y],       # ground truth label
                 "%.2f" %presm.max(),                # pre-softmax logits
                 "%.2f" % prob.max(),                # probabilistic softmax output  
                 "%s" %label_to_class_name[y_hat]    # predicted label
                ))

    for aidx, analyzer in enumerate(analyzers):
        # Measure execution time
        t_start = time.time()
        print('{} '.format(''.join(methods[aidx][-1])), end='', flush=True)
        
        is_input_analyzer = methods[aidx][0] == "input"
        # Analyze.
        if analyzer != None:
            a = analyzer.analyze(image if is_input_analyzer else x)
            t_elapsed = time.time() - t_start
            print('({:.4f}s) '.format(t_elapsed), end='', flush=True)
            # Postprocess.
            if not np.all(np.isfinite(a)):
                print("Image %i, analysis of %s not finite: nan %s inf %s" %
                      (i, methods[aidx][3],
                       np.any(np.isnan(a)), np.any(np.isinf(a))))
            if not is_input_analyzer:
                a = imgnetutils.postprocess(a, color_conversion, channels_first)
            a = methods[aidx][2](a)
            analysis[aidx] = a[0]
        else:
            print("Analyzer not available for this model.")
            analysis[aidx] = np.zeros_like(image)
    print()
            
    
    # Clear session.
    if keras.backend.backend() == 'tensorflow':
        keras.backend.clear_session()
    
    analysis_all.append(analysis)

Analyzing vgg16...
Input (0.0201s) Gradient (0.3347s) Integrated Gradients (2.4653s) Deconvnet (0.4642s) Guided Backprop (0.1525s) PatterNet (3.9709s) Pattern Attribution (5.8157s) LRP Epsilon (0.7294s) LRP-PresetAFlat (1.0880s) LRP-PresetBFlat (1.4424s) 
Analyzing resnet50...
Input (0.1504s) Gradient (2.3391s) Integrated Gradients (8.4948s) Deconvnet (2.9734s) Guided Backprop (2.9394s) PatterNet Analyzer not available for this model.
Pattern Attribution Analyzer not available for this model.
LRP Epsilon (5.2389s) LRP-PresetAFlat (7.2437s) LRP-PresetBFlat (9.2995s) 
Analyzing inception_v3...
Input (0.2535s) Gradient (3.4629s) Integrated Gradients (18.4573s) Deconvnet (6.5985s) Guided Backprop (5.3657s) PatterNet Analyzer not available for this model.
Pattern Attribution Analyzer not available for this model.
LRP Epsilon (10.7862s) LRP-PresetAFlat (11.4236s) LRP-PresetBFlat (15.9015s) 
Analyzing densenet121...
Input (0.4154s) Gradient (4.6482s) Integrated Gradients (25.3047s) Deconvnet 

In [ ]:
# Plot the analysis.
grid = [[analysis_all[i][j] for j in range(len(methods))]
        for i in range(n_nets)]
label, presm, prob, pred = zip(*text)
row_labels_left = [(n,'') for n in netnames]
row_labels_right = [('label: {}'.format(label[i]),'pred: {}'.format(pred[i])) for i in range(len(label))]
col_labels = [method[3] for method in methods]

eutils.plot_image_grid(grid, row_labels_left, row_labels_right, col_labels)